# 규칙 기반 학습을 이용한 Tic-Tac-Toe 인공지능
---

## Remind : 규칙 기반 학습 이란?
---
- 주어진 입력에 대해서 결과값을 도출하는 방법으로 if-then 방식이라고도 합니다.
- 확고한 규칙에 따라 학습 및 예측하는 방법입니다.

## Tic-Tac-Toe 게임이란?
---
1. 3x3 보드가 존재합니다.
2. 'O'와'X' 두개의 말이 있습니다.
3. 두 명의 사용자는 번갈아 가면서 말을 둡니다.
4. 한 명의 사용자가 가로, 세로, 대각선에 일렬로 두게 되면 승리합니다.
5. 무승부가 가능합니다.

## Tic-Tac-Toe 게임 만들기
---

우리는 Tic Tac Toe 게임을 직접 만들어 보겠습니다.

만들면서 지난 파이썬 응용 강의를 기억하도록 하겠습니다.

Tic Tac Toe 게임 보드는 하나의 클래스로서 작동하게됩니다.

In [1]:
import os
import numpy as np
from sys import platform
from IPython.display import clear_output

In [2]:
class GameBoard(object):
    def __init__(self):
        self.board_state = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
        self.board_size  = len(self.board_state)*len(self.board_state[0])
        self.board_rows  = len(self.board_state)
        self.board_cols  = len(self.board_state[0])
        self.marker      = 1
        self.result      = 0
        
    def print_board(self):
        for i, row in enumerate(self.board_state):
            for idx, col in enumerate(row):
                print(f" {self.draw_mark(col)} ", end="")
                if not (idx == (len(row)-1)):
                    print("|", end="")
                else:
                    print()
            if not (i == (len(self.board_state)-1)):
                print("---|---|---")
                
    def play_mark(self, inputs):

        if not inputs in list(range(self.board_size)):
            return f"Please play right places!"
        else:
            r, c = divmod(inputs, self.board_rows)
            if self.board_state[r][c] != 0:
                return f"Can not play that place!"
            else:
                self.board_state[r][c] = self.marker
        self.marker *= -1
        return 0
                
    def draw_mark(self, state):
        if state == -1:
            return 'X'
        elif state == 1:
            return 'O'
        else:
            return ' '
    def make_log(self, state=0):
        r"""
        State 0 : Draw
        State 1 : X player win
        State 2 : O player win
        """
        if state == 1:
            return "Player O win..."
        elif state == 2:
            return "Player X win..."
        else:
            return "Draw!"
        
    def check_vic(self):
        # Check Draw
        if not 0 in self.board_state:
            return self.make_log()
        # Check Rows
        for row in self.board_state:
            if sum(row) == 3:
                return self.make_log(1)
            elif sum(row) == -3:
                return self.make_log(2)
            
        # Check Columns
        for col in self.board_state.T:
            if sum(col) == 3:
                return self.make_log(1)
            elif sum(col) == -3:
                return self.make_log(2)
            
        # Check diagonal
        di1 = np.trace(self.board_state)
        di2 = np.trace(self.board_state[..., ::-1])
        
        if di1 == 3 or di2 == 3:
            return self.make_log(1)
        elif di1 == -3 or di2 == -3:
            return self.make_log(2)
        
        return 0
            
        
        
    def clear_output(self):
#         Windows
#         os.system( "cls")
#         Linux
#         os.system("clear")
#         Notebook
        clear_output()
    
    def play(self):
        while True:
            self.clear_output()
            self.print_board()
            
            # Print Problem or Information    
            if self.result:
                print(self.result)
                
            # Check Who win the game
            self.result = self.check_vic()
            if self.result:
                print(self.result)
                break            
            
            # Input user's play
            inputs = int(input(f"Player {self.draw_mark(self.marker)} : "))
            
            # Check it can be played?
            self.result = self.play_mark(inputs)
            if self.result:
                continue
            
            
            

## Play 해보기
---

직접 플레이를 해보겠습니다.

In [3]:
game = GameBoard()
game.play()

 O | X | O 
---|---|---
 X | O | X 
---|---|---
 O |   |   
Player O win...


## 규칙 기반 AI 만들기
---

우리는 우리가 만든 Tic-Tac-Toe 게임을 자동으로 플레이 하는 프로그램을 만들 수 있습니다.

우리가 만들어볼 방식은 규칙 기반으로 동작하는 상대입니다.

우리는 기존의 다른 코딩 방식처럼 동작을 완벽하게 이해한 후 정해진 절차에 따라 자동으로 말을 놓는 상대방을 제작할 수 있을 것입니다.

이런 방식을 규칙 기반 방식이라고 부릅니다.

이를 위해선 우리는 Tic-Tac-Toe 게임을 명확하게 이해할 필요가 있습니다.

이 게임을 이기기 위해서 어디에 두는 것이 가장 좋을까요?

확률에 기반하여 이길 수 있는 확률이 높은 곳을 기준으로 말을 둔다면 좋을 것이라는 가정하에 규칙을 제작하겠습니다.


===============  __Tic__ __Tac__ __Toe__ AI 게임 규칙===============

1. 이길 수 있는 곳이 있으면, 이길 수 있는 곳에 둔다.
2. 상대방이 이길 수 있는 곳이 있으면, 상대방이 이길 수 있는 곳에 둔다.
3. 가운데가 비어 있으면, 가운데에 둔다.
4. 모서리가 비어있다면, 모서리에 둔다.
5. 각 변 중 한곳이 비어있으면 둔다.

====================================================

이런 규칙들의 모음을 규칙 집합이라고 표현합니다.

우리는 이 5가지의 규칙 집합을 이용하여 자동으로 플레이하는 AI를 만들겠습니다.

In [4]:
from functools import partial, reduce
class RuleBasedAI(object):
    def __init__(self, board, marker=-1):
        self.board      = board
        self.board_size = len(board) * len(board[0])
        self.rows       = len(board)
        self.cols       = len(board[0])
        
    
        self.edges      = [[0,0], [0,2], [2, 0], [2, 2]]
        self.sides      = [[0,1], [1,0], [1, 1], [2, 1]]
        self.check_lst  = [partial(self.check_win, marker), 
                           partial(self.check_win, marker*-1),
                           self.check_middle,
                           self.check_edge, 
                           self.check_side]
    
    def view_board(self, board):
        self.board = board
        
    def calc_return(self, r, c):
        return (self.rows * r) + c
    
    def check_middle(self):
        r, c = self.rows//2, self.cols//2
        if self.board[self.rows//2][self.cols//2] == 0:
            return self.calc_return(r, c)
        return -1
    
    def check_edge(self):
        for r,c in self.edges:
            if self.board[r][c] == 0:
                return self.calc_return(r,c)
        return -1
        
    def check_side(self):
        for r,c in self.sides:
            if self.board[r][c] == 0:
                return self.calc_return(r,c)
        return -1
    
    def check_win(self, check_marker):
        # Check Rows
        for r, row in enumerate(self.board):
            if sum(row) == (2 * check_marker):
                for c, col in enumerate(row):
                    if col == 0:
                        return self.calc_return(r,c)
            
        # Check Columns
        for c, col in enumerate(self.board.T):
            if sum(col) == (2 * check_marker):
                for r, row in enumerate(col):
                    if row == 0:
                        return self.calc_return(r,c)

            
        # Check diagonal
        di1 = np.trace(self.board)
        di2 = np.trace(self.board[..., ::-1])
        
        if di1 == (2 * check_marker):
            for idx, val in enumerate(np.diag(self.board)):
                if val == 0 :
                    return self.calc_return(idx,idx)
                
        elif di2 == (2 * check_marker):
            for idx, val in enumerate(np.diag(self.board[..., ::-1])):
                print(val)
                print(idx)
                if val == 0 :
                    return self.calc_return(idx, (self.cols -1 - idx))
        return -1
    
    def __call__(self, board):
        self.view_board(board)
        
        for func in self.check_lst:
            result = func()
            if result != -1:
                return result
            

## 자동 플레이어 테스트
---

우리가 코딩한 자동 플레이어가 잘 동작하는지 프로그래밍 해봅시다.

In [5]:
Test_Board = np.array([[1,0,0],
                       [0,0,0],
                       [0,0,0]]) #맵을 바꾸어보세요
AI = RuleBasedAI(Test_Board)
AI(Test_Board)

4

## 규칙 기반 AI가 들어간 게임 보드 만들기
---

이제 처음 만들었던 플레이 보드와 AI를 합칠 차례입니다.

기본적인 동작은 처음 만들었던 게임 보드와 같습니다.

따라서 게임 보드를 상속하여 진행할 예정입니다.

In [6]:
class GAME_with_AI1(GameBoard):
    def __init__(self, marker = -1):
        super().__init__()
        self.AI_Marker = marker
        self.AI        = RuleBasedAI(self.board_state, self.AI_Marker)
        
    def __call__(self):
        while True:
            self.clear_output()
            self.print_board()
            
            # Check Who win the game
            self.win = self.check_vic()
            if self.win:
                print(self.win)
                break    
            
            if self.marker == self.AI_Marker:
                AI_inputs = self.AI(self.board_state)
                self.play_mark(AI_inputs)
                
            else:
                # Print Problem or Information    
                if self.result:
                    print(self.result)
                    
                # Input user's play
                inputs = int(input(f"Player {self.draw_mark(self.marker)} : "))

                # Check it can be played?
                self.result = self.play_mark(inputs)
                if self.result:
                    continue
    

## AI와 게임해보기
---

우리가 만든 규칙 기반 방식의 인공지능과 게임을 해봅시다!

이길 수 있나요?

있다면 어떻게 이길 수 있나요?

In [ ]:
game = GAME_with_AI1()
game()

   |   |   
---|---|---
   |   |   
---|---|---
   |   |   


## 규칙 기반 학습
---

규칙 기반 학습은 말 그대로 규칙 집합의 규칙들을 변경 혹은 추가하여 개선하는 것을 의미합니다.

Human Learning이라고도 할 수 있는데, 개발자가 규칙의 문제점을 찾고 이를 개선합니다.

개선하는 방법 혹은 규칙을 정의 하는 방법이 여러가지 있으나, 지금은 사람이 직접 경우의 수를 살펴 보는 것으로 개선한다고 가정하겠습니다.

현재 우리는 승리 확률에 기반하여 규칙 집합을 정의했고, 거의 완벽한 tic-tac-toe인공지능을 만들었습니다.

하지만, 사용자가 첫 수에 모서리 부분에 말을 둔다면, AI는 규칙에 의거하여 가운데에 두게 될 것입니다.

아래 그림과 같은 진행은 우리가 원하는 진행이 아닙니다.

| 첫번째 수 | 두번째 수 | 세번째 수 | 결과 |
| :---: | :---: | :---: | :---: |
| <img src="img/Chapter04/img1.png" width="100px" height="50px" title="img1"/> | <img src="img/Chapter04/img2.png" width="100px" height="50px" title="img2"/> | <img src="img/Chapter04/img3.png" width="100px" height="50px" title="img3"/> | <img src="img/Chapter04/img4.png" width="100px" height="50px" title="img4"/> |

이를 개선하기 위해 우리는 규칙 집합을 아래와 같이 개선할 수 있습니다.



===============  __Tic__ __Tac__ __Toe__ AI 게임 규칙 ==================

1. 이길 수 있는 곳이 있으면, 이길 수 있는 곳에 둔다.
2. 상대방이 이길 수 있는 곳이 있으면, 상대방이 이길 수 있는 곳에 둔다.
3. 첫번째 차례이고, 가운데가 비어 있으면, 가운데에 둔다.
4. 두번째 차례이고, 상대방이 모서리에 두었다면, 반대쪽 모서리에 둔다.
5. 모서리가 비어있다면, 모서리에 둔다.
6. 각 변 중 한곳이 비어있으면 둔다.

====================================================


이를 구현하기 위해 기존의 규칙 기반 모델을 상속받아 개선해보겠습니다.


In [25]:
class BetterRuleAI(RuleBasedAI):
    def __init__(self, board, marker=-1):
        super().__init__(board, marker=-1)
        self.check_lst  = [partial(self.check_win, marker), 
                           partial(self.check_win, marker*-1),
                           self.check_first,
                           self.check_edge, 
                           self.check_side]
        
    def check_first(self):
        now_board = reduce(lambda a,b:a+b, [self.board[r][c] for r,c in self.edges])
        if self.board.sum() == 0 or now_board == 0:
            return self.check_middle()

        elif now_board == 1:
            for r, c in self.edges:
                if self.board[r][c] == 1:
                    return self.calc_return(abs(r-2), abs(c-2))
        
        return -1

## 자동 플레이어 테스트
---

만든 AI가 잘 작동하는지 테스트 해봅시다.

In [26]:
Test_Board = np.array([[0,0,0],
                       [0,0,0],
                       [1,0,0]]) #맵을 바꾸어보세요
AI2 = BetterRuleAI(Test_Board)
AI2(Test_Board)

2

## 최종 AI 연결
---

이렇게 만들어진 규칙 기반 AI를 연결합니다.

우리는 이런 AI를 규칙 기반 AI라고 합니다.

혹은 이러한 규칙 기반 지능형 시스템을 일컬어 전문가 시스템이라고 부릅니다.

전문가 시스템(專門家 system, experts system)은 생성시스템의 하나로서, 딥러닝 이전 인공지능 기술의 응용분야 중에서 가장 활발하게 응용되고 있는 분야였습니다.

즉 인간이 특정분야에 대하여 가지고 있는 전문적인 지식을 정리하고 표현하여 컴퓨터에 기억시킴으로써, 일반인도 이 전문지식을 이용할 수 있도록 하는 시스템이다. 

의료 진단 시스템, 설계 시스템 등에서 활발하게 사용되었습니다.

In [27]:
class GAME_with_AI2(GAME_with_AI1):
    def __init__(self, marker=-1):
        super().__init__(marker = -1)
        self.AI        = BetterRuleAI(self.board_state, self.AI_Marker)

In [28]:
game2 = GAME_with_AI2()
game2()

 O | X | X 
---|---|---
 X | O | O 
---|---|---
 O | O | X 
Draw!
